##Setup



In [ ]:
!pip install -q pytorch-lightning
!pip install -q transformers

     |████████████████████████████████| 829kB 9.4MB/s 
     |████████████████████████████████| 112kB 25.5MB/s 
     |████████████████████████████████| 829kB 14.4MB/s 
     |████████████████████████████████| 276kB 51.4MB/s 
     |████████████████████████████████| 1.3MB 47.9MB/s 
     |████████████████████████████████| 143kB 52.1MB/s 
     |████████████████████████████████| 296kB 50.0MB/s 
     |████████████████████████████████| 1.9MB 13.0MB/s 
     |████████████████████████████████| 890kB 51.4MB/s 
     |████████████████████████████████| 3.2MB 47.6MB/s 


In [ ]:
import transformers
from torch.utils.data import DataLoader, TensorDataset, random_split, RandomSampler, Dataset
import pandas as pd
import numpy as np

import torch.nn.functional as F
import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import ModelCheckpoint

from transformers import AutoModelWithLMHead, AutoTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import math
import random
import re
import argparse
from tqdm import tqdm, trange
import io

In [ ]:
from google.colab import files
uploaded = files.upload()
uploaded1 = files.upload()
uploaded2 = files.upload()

Saving dict.txt to dict.txt


Saving sciryl.csv to sciryl.csv


##BART Part

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'BART/'

Mounted at /content/gdrive


In [ ]:
class LitModel(pl.LightningModule):
  # Instantiate the model
  def __init__(self, learning_rate, tokenizer, model, hparams):
    super().__init__()
    self.tokenizer = tokenizer
    self.model = model
    self.learning_rate = learning_rate
    self.hparams = hparams

    if self.hparams.freeze_encoder:
      freeze_params(self.model.get_encoder())

    if self.hparams.freeze_embeds:
      self.freeze_embeds()
  
  def freeze_embeds(self):
    ''' freeze the positional embedding parameters of the model; adapted from finetune.py '''
    freeze_params(self.model.model.shared)
    for d in [self.model.model.encoder, self.model.model.decoder]:
      freeze_params(d.embed_positions)
      freeze_params(d.embed_tokens)

  # Do a forward pass through the model
  def forward(self, input_ids, **kwargs):
    return self.model(input_ids, **kwargs)
  
  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr = self.learning_rate)
    return optimizer

  def training_step(self, batch, batch_idx):
    # Load the data into variables
    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]
    # Shift the decoder tokens right (but NOT the tgt_ids)
    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)

    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]
    # Create the loss function
    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    # Calculate the loss on the un-shifted tokens
    loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))
    return {'loss':loss}

  def validation_step(self, batch, batch_idx):

    src_ids, src_mask = batch[0], batch[1]
    tgt_ids = batch[2]

    decoder_input_ids = shift_tokens_right(tgt_ids, tokenizer.pad_token_id)
    
    # Run the model and get the logits
    outputs = self(src_ids, attention_mask=src_mask, decoder_input_ids=decoder_input_ids, use_cache=False)
    lm_logits = outputs[0]

    ce_loss_fct = torch.nn.CrossEntropyLoss(ignore_index=self.tokenizer.pad_token_id)
    val_loss = ce_loss_fct(lm_logits.view(-1, lm_logits.shape[-1]), tgt_ids.view(-1))

    return {'loss': val_loss}
  
  # Method that generates text using the BartForConditionalGeneration's generate() method
  def generate_text(self, text, eval_beams, early_stopping = True, max_len = 40, startT = None):
    ''' Function to generate text '''
    if startT == None:
      dstartT = self.tokenizer.pad_token_id
    else:
      dstartT = tokenizer.convert_tokens_to_ids(startT)
    generated_ids = self.model.generate(
        text["input_ids"],
        attention_mask=text["attention_mask"],
        use_cache=True,
        decoder_start_token_id = dstartT,
        num_beams= eval_beams,
        max_length = max_len,
        early_stopping = early_stopping
    )
    return [self.tokenizer.decode(w, skip_special_tokens=True, clean_up_tokenization_spaces=True) for w in generated_ids]

def freeze_params(model):
  ''' Function that takes a model as input (or part of a model) and freezes the layers for faster training
      adapted from finetune.py '''
  for layer in model.parameters():
    layer.requires_grade = False


In [ ]:
# Create a dataloading module as per the PyTorch Lightning Docs
class SummaryDataModule(pl.LightningDataModule):
  def __init__(self, tokenizer, data_file, batch_size, num_examples = 20000):
    super().__init__()
    self.tokenizer = tokenizer
    self.data_file = data_file
    self.batch_size = batch_size
    self.num_examples = num_examples
  
  # Loads and splits the data into training, validation and test sets with a 60/20/20 split
  def prepare_data(self):
    self.data = pd.read_csv(self.data_file)[:self.num_examples]
    self.train, self.validate, self.test = np.split(self.data.sample(frac=1), [int(.6*len(self.data)), int(.8*len(self.data))])

  # encode the sentences using the tokenizer  
  def setup(self, stage):
    self.train = encode_sentences(self.tokenizer, self.train['source'], self.train['target'])
    self.validate = encode_sentences(self.tokenizer, self.validate['source'], self.validate['target'])
    self.test = encode_sentences(self.tokenizer, self.test['source'], self.test['target'])

  # Load the training, validation and test sets in Pytorch Dataset objects
  def train_dataloader(self):
    dataset = TensorDataset(self.train['input_ids'], self.train['attention_mask'], self.train['labels'])                          
    train_data = DataLoader(dataset, sampler = RandomSampler(dataset), batch_size = self.batch_size)
    return train_data

  def val_dataloader(self):
    dataset = TensorDataset(self.validate['input_ids'], self.validate['attention_mask'], self.validate['labels']) 
    val_data = DataLoader(dataset, batch_size = self.batch_size)                       
    return val_data

  def test_dataloader(self):
    dataset = TensorDataset(self.test['input_ids'], self.test['attention_mask'], self.test['labels']) 
    test_data = DataLoader(dataset, batch_size = self.batch_size)                   
    return test_data



In [ ]:
hparams = argparse.Namespace()

hparams.freeze_encoder = True
hparams.freeze_embeds = True
hparams.eval_beams = 4

In [ ]:
def shift_tokens_right(input_ids, pad_token_id):
  """ Shift input ids one token to the right, and wrap the last non pad token (usually <eos>).
      This is taken directly from modeling_bart.py
  """
  prev_output_tokens = input_ids.clone()
  index_of_eos = (input_ids.ne(pad_token_id).sum(dim=1) - 1).unsqueeze(-1)
  prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
  prev_output_tokens[:, 1:] = input_ids[:, :-1]
  return prev_output_tokens

def encode_sentences(tokenizer, source_sentences, target_sentences, max_length=32, pad_to_max_length=True, return_tensors="pt"):
  ''' Function that tokenizes a sentence 
      Args: tokenizer - the BART tokenizer; source and target sentences are the source and target sentences
      Returns: Dictionary with keys: input_ids, attention_mask, target_ids
  '''

  input_ids = []
  attention_masks = []
  target_ids = []
  tokenized_sentences = {}

  for sentence in source_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

  input_ids = torch.cat(input_ids, dim = 0)
  attention_masks = torch.cat(attention_masks, dim = 0)

  for sentence in target_sentences:
    encoded_dict = tokenizer(
          sentence,
          max_length=max_length,
          padding="max_length" if pad_to_max_length else None,
          truncation=True,
          return_tensors=return_tensors,
          add_prefix_space = True
      )
    # Shift the target ids to the right
    # shifted_target_ids = shift_tokens_right(encoded_dict['input_ids'], tokenizer.pad_token_id)
    target_ids.append(encoded_dict['input_ids'])

  target_ids = torch.cat(target_ids, dim = 0)
  

  batch = {
      "input_ids": input_ids,
      "attention_mask": attention_masks,
      "labels": target_ids,
  }

  return batch


def noise_sentence(sentence_, percent_words, replacement_token = "<mask>"):
  '''
  Function that noises a sentence by adding <mask> tokens
  Args: sentence - the sentence to noise
        percent_words - the percent of words to replace with <mask> tokens; the number is rounded up using math.ceil
  Returns a noised sentence
  '''
  # Create a list item and copy
  sentence_ = sentence_.split(' ')
  sentence = sentence_.copy()
  
  num_words = math.ceil(len(sentence) * percent_words)
  
  # Create an array of tokens to sample from; don't include the last word as an option because in the case of lyrics
  # that word is often a rhyming word and plays an important role in song construction
  sample_tokens = set(np.arange(0, np.maximum(1, len(sentence)-1)))
  
  words_to_noise = random.sample(sample_tokens, num_words)
  
  # Swap out words, but not full stops
  for pos in words_to_noise:
      if sentence[pos] != '.':
          sentence[pos] = replacement_token
  
  # Remove redundant spaces
  sentence = re.sub(r' {2,5}', ' ', ' '.join(sentence))
  
  # Combine concurrent <mask> tokens into a single token; this just does two rounds of this; more could be done
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  sentence = re.sub(r'<mask> <mask>', "<mask>", sentence)
  return sentence
  

In [ ]:
# Load the model
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig, BartModel

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base', add_prefix_space=True)

bart_model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-base")


In [ ]:
# Load the data into the model for training
summary_data = SummaryDataModule(tokenizer, '/content/gdrive/My Drive/BART learns to rap/lyrics_simple_noised.csv',
                                 batch_size = 16, num_examples = 140000)

# Load the model from a pre-saved checkpoint; alternatively use the code below to start training from scratch
# model = LitModel.load_from_checkpoint(base_dir + "checkpoint_files_2/8_ep_140k_simple_0210.ckpt",
#                                       learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

model = LitModel(learning_rate = 2e-5, tokenizer = tokenizer, model = bart_model, hparams = hparams)

In [ ]:
#checkpoint = ModelCheckpoint(filepath=base_dir + 'checkpoint_files_2/')
trainer = pl.Trainer(gpus = 1,
                     max_epochs = 2,
                     min_epochs = 2,
                     auto_lr_find = False,
                     
                     progress_bar_refresh_rate = 500)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


In [ ]:
# Fit the instantiated model to the data
trainer.fit(model, summary_data)


  | Name  | Type                         | Params
-------------------------------------------------------
0 | model | BartForConditionalGeneration | 139 M 
-------------------------------------------------------
139 M     Trainable params
0         Non-trainable params
139 M     Total params
557.682   Total estimated model params size (MB)


1

In [ ]:
def generate_lyrics(seed_line, num_lines, model_, startW, noise_percent = 0.25, multiple_lines = False, max_line_history = 3):
  ''' Function that generates lyrics based on previously generated lyrics 
      Args: seed_line - a line to start off the machine
            num_lines - the number of lines to generate
            model_ - the model used to generate the text
            multiple_lines - whether the model generates based on multiple previous lines or just the past line
            max_line_history - the maximum number of previous lines used in the current input
      Returns a list with num_lines of rap lines
  '''
  # Put the model on eval mode
  model_.to(torch.device('cpu'))
  model_.eval()
  lyrics = []
  lyrics.append(seed_line)
  prompt_line_tokens = tokenizer(noise_sentence(seed_line, 0.2), max_length = 32, return_tensors = "pt", truncation = True)
  # Loop through the number of lines generating a new line based on the old

  line = [seed_line]
  for i in range(num_lines):
    # Print out the new line
    print(line[0].strip())
    lyrics.append(line[0])
    line = model.generate_text(prompt_line_tokens, eval_beams = 4, startT = startW)
    # This deals with an artefact in the training data that I had an issue cleaning
    if line[0].find(":") != -1:
      line[0] = re.sub(r'[A-Z]+: ', '', line[0])
    # This allows the model to generate a new line conditioned on more than one line
    if multiple_lines:
      start_line = np.maximum(0, i - max_line_history)
      end_line = i
      prompt_line = ' '.join(lyrics[start_line:end_line]) # Going to end_line is fine because it is non-inclusive
    else:
      prompt_line = lyrics[i]
    prompt_line_tokens = tokenizer(noise_sentence(prompt_line, noise_percent), max_length = 32, return_tensors = "pt", truncation = True)

  return lyrics

# LSTM


In [ ]:
BATCH_SIZE = 32

SUBSET = 1000
MIN_WORD_FREQUENCY = 10
SEQUENCE_LEN = 5
EPOCH = 15

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving fusion.txt to fusion.txt


In [ ]:
#songdata_UTF8_clean.txt - there were some qoutes in the file that were removed (also double space replaced by one) 
# then the lyric column were extracted and converted to the utf8 format

#open file and save it in to list 
with open('fusion.txt', 'r') as content_file:
    content = content_file.read()
songlyrics = content.split("\"")


#cleaning - removing empty elements from the list
clean_songlyrics = []
for item in songlyrics:
  if len(item)>1:
    clean_songlyrics.append(item)
# print(clean_songlyrics[0])


#make a list(lyrics) of list(sentences)
lyrics = []
for l in clean_songlyrics:
#   l = l.replace("\n", "")
#   l = l.replace("\n", "")
  sentences = l.split("\n")
  lyrics.append(sentences)
  


#cleaning - removing empty elements from sentences of each lyric
    
lyrics = [[s for s in l if len(s)>1] for l in lyrics ]

In [ ]:
#cleaning - removing brackets from the main list (lyrics)
import re
regex = re.compile(".*?\[(.*?)\]")

bracket = []
for i in range(len(lyrics)):

  for j in range(len(lyrics[i])):

    result = re.findall(regex, lyrics[i][j])

    for item in result:

      #keep them in the bracket list
      if item not in bracket:
        bracket.append(item)

      #remove the brackets from the sentences
      lyrics[i][j] = lyrics[i][j].replace("["+item+"]", "")
    
    
  #cleaning- remove empty elements (sentences) appeared on each lyric
  lyrics[i] = [s for s in lyrics[i] if len(s)>1]

['suck my cock']


In [ ]:
print(len(lyrics))
lyrics = lyrics[0:SUBSET]
print(len(lyrics))

6251
1000


In [ ]:
for i in range(len(lyrics)):
  for j in range(len(lyrics[i])):
    lyrics[i][j] += "\n"
    
print(lyrics[0])

['Chickens in the sink\n', 'Crack on the stove\n', 'Triple beam for the weight\n', 'Cut the middle man tryna get 8 white bitches\n', 'Snorting off the plate coke from Colombia\n', 'Riding round in the Benz looking like a trap star\n', 'Flipping raw day and night\n', "Tryna get my money right naw I ain't living rite\n", "Fucking bitches all night kick 'em out the next day\n", 'Everyday is pay day\n', 'Hit me with a half bring it back in two days\n', 'Living in Cali by the ocean\n', 'Condo got em open\n', 'Long dick stroke em\n', "I send 'em to the husband he over there fussing\n", "I'm over here fucking no loving\n", "Puffing on heavenly rock star 70's\n", 'Slim pants ripped jeans Jeremy Scott wings\n', 'Fucking heavily counting up cream\n', 'Rolling on beans disobedient with trees\n', 'Tatted from neck to ear bitches they want to stare\n', 'Macking with hoes as I pose\n', 'Hand in my draws henny out my pores\n', 'As I pour hit the blunt too much you on the dash as I pass\n', '(Hook)\n'

In [ ]:
#model - based on this link : https://medium.com/coinmonks/word-level-lstm-text-generator-creating-automatic-song-lyrics-with-neural-networks-b8a1617104fb

text_in_words = [w.lower()   for l in lyrics for s in l for w in s.split(' ') if len(w)>0]
# print(text_in_words[0:9])


#---------------Calculate word frequency

word_freq = {}
for word in text_in_words:
    word_freq[word] = word_freq.get(word, 0) + 1
    
ignored_words = set()
for k, v in word_freq.items():
    if word_freq[k] < MIN_WORD_FREQUENCY:
        ignored_words.add(k)
        
words = set(text_in_words)
print('Unique words before ignoring:', len(words))
print('Ignoring words with frequency <', MIN_WORD_FREQUENCY)
words = sorted(set(words) - ignored_words)
print('Unique words after ignoring:', len(words))

word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

Unique words before ignoring: 24676
Ignoring words with frequency < 10
Unique words after ignoring: 2173


In [ ]:
# ------------cut the text in semi-redundant sequences of SEQUENCE_LEN words
STEP = 1

sentences = []
next_words = []
ignored = 0
for i in range(0, len(text_in_words) - SEQUENCE_LEN, STEP):
    # Only add sequences where no word is in ignored_words
    if len(set(text_in_words[i: i+SEQUENCE_LEN+1]).intersection(ignored_words)) == 0:
        sentences.append(text_in_words[i: i + SEQUENCE_LEN])
        next_words.append(text_in_words[i + SEQUENCE_LEN])
    else:
        ignored = ignored+1
print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))

Ignored sequences: 153390
Remaining sequences: 63048


In [ ]:
#-------------Shuffle and split training set
import numpy as np

def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return x_train, y_train, x_test, y_test

  
sentences, next_words, sentences_test, next_words_test = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Size of training set = 61787
Size of test set = 1261


In [ ]:
#------------Building the model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Bidirectional, Embedding, InputLayer

model = Sequential()
model.add(Embedding(input_dim=len(words), output_dim=1024))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
model.add(Activation('softmax'))

In [ ]:
def generator(sentence_list, next_word_list, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, SEQUENCE_LEN), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size):
            for t, w in enumerate(sentence_list[index % len(sentence_list)]):
                x[i, t] = word_indices[w]
            y[i] = word_indices[next_word_list[index % len(sentence_list)]]
            index = index + 1
        yield x, y

In [ ]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    examples_file.write('\n----- Generating text after Epoch: %d\n' % epoch)

    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
        sentence = seed
        examples_file.write('----- Diversity:' + str(diversity) + '\n')
        examples_file.write('----- Generating with seed:\n"' + ' '.join(sentence) + '"\n')
        examples_file.write(' '.join(sentence))

        for i in range(50):
            x_pred = np.zeros((1, SEQUENCE_LEN, len(words)))
            for t, word in enumerate(sentence):
                x_pred[0, t, word_indices[word]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]

            sentence = sentence[1:]
            sentence.append(next_word)

            examples_file.write(" "+next_word)
        examples_file.write('\n')
    examples_file.write('='*80 + '\n')
    examples_file.flush()

In [ ]:
#-----------------training model

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'])


from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
file_path = "./checkpoints/LSTM_LYRICS-epoch{epoch:03d}-words%d-sequence%d-minfreq%d-" \
                "loss{loss:.4f}-acc{acc:.4f}-val_loss{val_loss:.4f}-val_acc{val_acc:.4f}" % \
                (len(words), SEQUENCE_LEN, MIN_WORD_FREQUENCY)

checkpoint = ModelCheckpoint(file_path, monitor='val_acc', save_best_only=True)
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=5)
callbacks_list = [checkpoint, print_callback, early_stopping]

examples_file = open("examples.txt", "w")



# BATCH_SIZE = 32
model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=EPOCH, #100
                        validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/15
1931/1931 [==============================] - 210s 107ms/step - loss: 6.0472 - accuracy: 0.0736 - val_loss: 4.5927 - val_accuracy: 0.2211
Epoch 2/15
1931/1931 [==============================] - 201s 104ms/step - loss: 4.3548 - accuracy: 0.2449 - val_loss: 4.0154 - val_accuracy: 0.3125
Epoch 3/15
1931/1931 [==============================] - 200s 104ms/step - loss: 3.5422 - accuracy: 0.3480 - val_loss: 3.7744 - val_accuracy: 0.3656
Epoch 4/15
1931/1931 [==============================] - 208s 108ms/step - loss: 2.9662 - accuracy: 0.4174 - val_loss: 3.7090 - val_accuracy: 0.3906
Epoch 5/15
1931/1931 [==============================] - 207s 107ms/step - loss: 2.4910 - accuracy: 0.4808 - val_loss: 3.6965 - val_accuracy: 0.4055
Epoch 6/15
1931/1931 [==============================] - 206s 107ms/step - loss: 2.0766 - accuracy: 0.5477 - val_loss: 3.7155 - val_accuracy: 0.4141
Epoch 7/15
1931/1931 [==============================] - 205s 106ms/step - loss: 1.7327 - accuracy: 0.6109 - val_

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
sentence = ['i', 'like', 'you', 'much', 'more']

x_pred = np.zeros((SEQUENCE_LEN, len(words)))
# print(word_indices['i'])
for t, word in enumerate(sentence):
#   print("here")
  x_pred[t, word_indices[word]] = 1.
  


diversity  = 10
preds = model.predict(x_pred, verbose=0)[0]
next_index = sample(preds, diversity)
next_word = indices_word[next_index]
print(next_word)

some)



In [ ]:
seed =  ['you', 'and', 'me', 'forever']
sentence =  ['I']
  
for i in range(30):
  x_pred = np.zeros((SEQUENCE_LEN, len(words)))
  for t, word in enumerate(seed):
    x_pred[t, word_indices[word]] = 1.
  preds = model.predict(x_pred, verbose=0)[0]
  next_index = sample(preds, diversity)
  next_word = indices_word[next_index]
  sentence.append(next_word)
  seed = seed[1:]
  seed.append(next_word)
  if "\n" in next_word:
    break

print(sentence)

sent--['i', 'like', 'you', 'much', 'more', 'fucking']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with', 'part']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with', 'part', 'so\n']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with', 'part', 'so\n', 'juice\n']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with', 'part', 'so\n', 'juice\n', 'yes']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with', 'part', 'so\n', 'juice\n', 'yes', 'shooting']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with', 'part', 'so\n', 'juice\n', 'yes', 'shooting', 'do\n']
sent--['i', 'like', 'you', 'much', 'more', 'fucking', 'hot', 'that’s', 'with

features:

*   SUBSET = 1000
*   MIN_WORD_FREQUENCY = 15
*   SEQUENCE_LEN = 1 
*   EPOCH = 10

result:

a bunch of 'i' and 'and'

features:

*   SUBSET = 1000
*   MIN_WORD_FREQUENCY = 15
*   SEQUENCE_LEN = 3 --------------
*   EPOCH = 10

result:

a bunch of 'i' , 'and' , 'i'll' 

# Rhyming Part

In [ ]:

import pandas as pd

STRESSES = {'AA1', 'AE1', 'AH1', 'AO1', 'AW1', 'AY1', 'EH1', 'ER1', 'EY1', 'IH1', 'IY1', 'OW1', 'OY1', 'UH1', 'UW1'}
"""
def fileToDict(filepath):
    dictA = dict()
    with open(filepath) as fileA:
        for line in fileA:
            words = line.split() 
            dictA[words[0]] = tuple(words[1:])
    return dictA

PDICT = fileToDict("CMUdict.txt") 
print(len(PDICT.keys()))

import nltk 
nltk.download('words')
nltk.download('punkt')
file_content = open("fusion.txt").read()
tokens = nltk.word_tokenize(file_content)

def cleanDict (pDict):
  phodict = {} 
  for i in pDict.keys():
     if ((i.lower() in tokens) and (i.isalpha())):
       phodict[i] = pDict[i]
  return phodict
PHODICT = cleanDict (PDICT)
print(len(PHODICT.keys()))
"""
import ast
file = open("dict.txt", "r")

contents = file.read()
PHODICT = ast.literal_eval(contents)

file.close()

def isSubList(listA, listB):
    if len(listA) < len(listB):
        return isSubList(listB, listA)
    n = len(listB)
    for start in range(len(listA)-n+1):
        if all(listA[start+i] == listB[i] for i in range(n)):
            return True
    return False

def getRhymes(word):
    print(type(word))
    sounds = PHODICT[word]
    for index,sound in enumerate(reversed(sounds)):
        if sound in STRESSES:
            ending = sounds[-index-1:]
            break
    yielded = set()
    for wordB, soundsB in PHODICT.items():
        if (ending == soundsB[-index-1:]) and (soundsB not in yielded) and (not isSubList(sounds, soundsB)):
            yielded.add(soundsB)
            yield wordB



#Generation Part

In [ ]:
def rhymeFinder(inWord):
  lastWord = inWord
  sameRhyme = []
  for word in getRhymes(lastWord.upper()):
    sameRhyme.append(word.lower())
  outWord = random.choice(sameRhyme)
  return outWord

def reverseSentence(inList):
  words = inList[0].split()
  outStr = ' '.join(reversed(words))
  return outStr

def reverseW(line):
  words = line.split()
  reversed_words = ' '.join(reversed(words))
  return reversed_words

PADDING_TEXT = """They tryna be cray 
They tryna be cray 
She wanna meet Carti 
That bitch is a Barbie 
I'ma fuck these hoes 
I'm on 730
Got a brand new pack like Kid Cudi
I smoke dope like Kid Cudi
Push up and get the slugs from me  
"""
END = """<eod> </s> <eos>"""

TEXT_PD = reverseW(PADDING_TEXT) + reverseW(END)


In [ ]:
firstSeed = "You and me forever "
allLyrics = []

BART_sentence = generate_lyrics(seed_line = firstSeed, num_lines = 2, model_ = model, startW = None,
                           noise_percent = 0.25, multiple_lines = False, max_line_history = 1)
allLyrics.append(BART_sentence[-1])

startword = rhymeFinder(BART_sentence[-1].split()[-1])

prompt = " " + startword
tpmorp = reverseW(prompt)
XLinputs = XLtokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")
XLinputs = XLinputs.to(device)
prompt_length = len(XLtokenizer.decode(XLinputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
XLoutputs = XLmodel.generate(XLinputs, max_length=70, do_sample=True, top_p=0.99, top_k=25, repetition_penalty  = 6.0, length_penalty = 0.01)

XLgenerated = XLtokenizer.decode(XLoutputs[0])[prompt_length:] + prompt
allLyrics.append(XLgenerated)
print(allLyrics)

You and me forever
I don't give a fuck what you think
<class 'str'>
[" I don't give a fuck what you think", ' my own way wink']


In [ ]:
!pip install rake-nltk
from rake_nltk import Rake

r = Rake()
r.extract_keywords_from_sentences(allLyrics)
keywords = r.get_ranked_phrases()

keySentence = ''
for i in keywords:
  keySentence += i
  keySentence += ' '

In [ ]:
BART_sentence1 = generate_lyrics(seed_line = keySentence, num_lines = 2, model_ = model, startW = None,
                           noise_percent = 0.25, multiple_lines = False, max_line_history = 1)
allLyrics.append(BART_sentence1[-1])

startword = rhymeFinder(BART_sentence1[-1].split()[-1])

prompt = " " + startword
tpmorp = reverseW(prompt)
XLinputs = XLtokenizer.encode(PADDING_TEXT + prompt, add_special_tokens=False, return_tensors="pt")
XLinputs = XLinputs.to(device)
prompt_length = len(XLtokenizer.decode(XLinputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
XLoutputs = XLmodel.generate(XLinputs, max_length=70, do_sample=True, top_p=0.99, top_k=25, repetition_penalty  = 6.0, length_penalty = 0.01)

XLgenerated = XLtokenizer.decode(XLoutputs[0])[prompt_length:] + prompt
allLyrics.append(XLgenerated)
print(allLyrics)

way wink think give fuck
I don't give a fuck
<class 'str'>
[" I don't give a fuck what you think", ' my own way wink', " I don't give a fuck what you think", '... rink', " I don't give a fuck", '. I can tuck']
